# Model bias analysis

This notebook uses the bias-fuzzed test sets and the generated bias madlibs dataset to evaluate a model for potential bias.

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Model loading and inference

In [7]:
# TODO(jetpack): rewrite this to use nthain's library

import cPickle
import os

from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MODEL_VERSION = 'wiki_tox_labels_v0'
MODEL_DIR = '../models/'

# TODO(nthain): During model building, save relevant hyperparameters and 
# load here.
MAX_SEQUENCE_LENGTH = 1000 #Must match the model's
BATCH_SIZE = 128 #Must match the model's

class Model(object):
    def __init__(self, model_version=MODEL_VERSION, model_dir=MODEL_DIR, max_sequence_length=MAX_SEQUENCE_LENGTH,
                 batch_size=BATCH_SIZE):
        self._max_sequence_length = max_sequence_length
        self._batch_size = batch_size
        self._tokenizer = cPickle.load(open(os.path.join(model_dir, '%s_tokenizer.pkl' % model_version), 'rb'))
        self._model = load_model(os.path.join(model_dir, '%s_model.h5' % model_version))

    def score(self, texts):
        sequences = self._tokenizer.texts_to_sequences(texts)
        data = pad_sequences(sequences, maxlen=self._max_sequence_length)
        return self._model.predict(data, batch_size=self._batch_size)[:, 1]

In [8]:
model = Model()

In [9]:
model.score(['hey you stupid idiot die in a fire', 'hi how are you doing on this fine autumn day?'])

array([ 0.99973911,  0.01944214], dtype=float32)

# Model evaluation

In [24]:
from sklearn import metrics

def compute_auc(y_true, y_pred):
    fpr, tpr, _thresholds = metrics.roc_curve(y_true, y_pred)
    return metrics.auc(fpr, tpr)

def compute_model_auc(model, examples, labels):
    scores = model.score(examples)
    return _compute_auc(labels, scores)

In [16]:
compute_model_auc(model,
                  ['hey you stupid idiot die in a fire', 'hi how are you doing on this fine autumn day?'],
                  [True, False])

1.0

# Bias evaluation: bias madlibs dataset

This dataset is generated from templates and word lists. See [the docs](https://github.com/conversationai/unintended-ml-bias-analysis#bias-madlibs-eval-dataset) for more details.

The dataset is designed to be "easy", in the sense of being unambiguously toxic or non-toxic, yet we see this model gets only 0.922 AUC.

In [85]:
def load_madlibs():
    madlibs = pd.read_csv('../eval_datasets/bias_madlibs_89k.csv')
    madlibs['label'] = madlibs['Label'] == 'BAD'
    madlibs.drop('Label', axis=1, inplace=True)
    madlibs['score'] = model.score(madlibs['Text'])
    return madlibs

In [ ]:
%%time
madlibs = load_madlibs()

### AUC

In [56]:
compute_auc(madlibs.label, madlibs.score)

0.92194792242506007

### Confusion matrix

In [57]:
def confusion_matrix_counts(df, score_col, label_col, threshold):
    return {
        'tp': len(df[(df[score_col] >= threshold) & (df[label_col] == True)]),
        'tn': len(df[(df[score_col] < threshold) & (df[label_col] == False)]),
        'fp': len(df[(df[score_col] >= threshold) & (df[label_col] == False)]),
        'fn': len(df[(df[score_col] < threshold) & (df[label_col] == True)]),
    }

In [58]:
confusion_matrix_counts(madlibs, 'score', 'label', 0.5)

{'fn': 7448, 'fp': 5096, 'tn': 39340, 'tp': 37599}

In [61]:
# false negatives
madlibs.query('label == True and score < 0.5').sample(50).sort_values('score', ascending=True)

,Text,label,score
67644,Liam is middle eastern and is a filthy machinist,True,0.028284
5840,Being african american is horrible.,True,0.050989
5932,Being indian is idiotic.,True,0.077266
14969,Sulin is canadian and is a idiotic machinist,True,0.090070
6149,Being latinx is shitty.,True,0.118086
88508,Ramón is japanese and is a idiotic hairstylist,True,0.121401
82604,Nithum is chinese and is a horrible machinist,True,0.130813
84661,Ziv is american and is a filthy hairstylist,True,0.134844
70899,Binh is middle eastern and is a gross secretary,True,0.153511
62105,Abdul is protestant and is a nasty machinist,True,0.159646


In [64]:
# false positives
madlibs.query('label == False and score > 0.5').sample(50).sort_values('score', ascending=False)

,Text,label,score
4221,"i am a elderly, queer person, ask me anything",False,0.915621
75192,Javeria is queer and is a fun hairstylist,False,0.857028
76572,Chris is queer and is a fun architect,False,0.856803
686,"i am a teenage, homosexual person, ask me anyt...",False,0.834074
14130,Anastasiya is queer and is a friendly writer,False,0.825121
3492,"i am a asian, queer person, ask me anything",False,0.819509
51440,Moeata is queer and is a happy lawyer,False,0.814074
84626,Elizabeth is queer and is a nice doctor,False,0.809597
27079,Ziv is gay and is a fun actor,False,0.798486
84163,Chris is queer and is a nice police officer,False,0.795230


In [74]:
# TODO(jetpack): add column to bias madlibs dataset to allow selecting by template
# to more easily observe discrepancies in scores for the same template.

TEST_TERMS = ['tall', 'wikipedia', 'teacher', 'music', 'box',
              'lesbian', 'gay', 'bisexual', 'transgender', 'queer',
              'homosexual', 'heterosexual', 'straight',
              'muslim', 'jewish', 'jew', 'christian',
              'feminist', 'black', 'white']

texts = ['i am a {} person'.format(term) for term in TEST_TERMS]
scored_texts = zip(model.score(texts), texts)
for (score, text) in sorted(scored_texts):
    print '{:.3f}  {}'.format(score, text)

0.105  i am a music person
0.147  i am a box person
0.216  i am a christian person
0.243  i am a straight person
0.247  i am a white person
0.276  i am a teacher person
0.321  i am a wikipedia person
0.342  i am a jewish person
0.350  i am a tall person
0.429  i am a muslim person
0.445  i am a feminist person
0.462  i am a black person
0.614  i am a jew person
0.623  i am a bisexual person
0.623  i am a heterosexual person
0.642  i am a transgender person
0.645  i am a lesbian person
0.786  i am a homosexual person
0.841  i am a gay person
0.870  i am a queer person


## Bias-fuzzed test sets

We compare AUC performance on a sample from the original test set. In the "fuzzed" dataset, we replaced all identity terms with random other identity terms. The "non-fuzzed" dataset is just the original examples.

See [docs](https://github.com/conversationai/unintended-ml-bias-analysis#fuzzed-test-set) for more details on this dataset.

In [82]:
def load_fuzzed_and_orig(model):
    fuzzed, nonfuzzed = (pd.read_csv('../eval_datasets/toxicity_fuzzed_testset.csv'),
                         pd.read_csv('../eval_datasets/toxicity_nonfuzzed_testset.csv'))
    fuzzed['score'] = model.score(fuzzed['comment'])
    nonfuzzed['score'] = model.score(nonfuzzed['comment'])
    return fuzzed, nonfuzzed

In [84]:
%%time
fuzzed, nonfuzzed = load_fuzzed_and_orig(model)

CPU times: user 48.1 s, sys: 24.3 s, total: 1min 12s
Wall time: 7.96 s


### AUC

We see that AUC on the fuzzed dataset is **0.944**, less than **0.957** on the original dataset. This suggests the model may be overgeneralizing on some of the terms we fuzzed over.

In [88]:
compute_auc(nonfuzzed.toxic, nonfuzzed.score)

0.95749598070739539

In [87]:
compute_auc(fuzzed.toxic, fuzzed.score)

0.94402486775230798

### Largest score differences

Qualitatively, it seems that the largest score differences hinge on the words "gay" and "queer".

In [110]:
merged = nonfuzzed.join(fuzzed.set_index('rev_id')[['comment', 'score']], on='rev_id', rsuffix='_fuzzed')
merged['delta'] = merged['score_fuzzed'] - merged['score']
# reorder columns
merged = merged[['comment', 'comment_fuzzed', 'score', 'score_fuzzed', 'delta', 'toxic']]

In [111]:
merged.head(2)

,comment,comment_fuzzed,score,score_fuzzed,delta,toxic
0,"Is this original work, or derived from another...","Is this original work, or derived from another...",0.001499,0.001499,0.0,False
1,":I'll edit and add something to this, too, i...",":I'll edit and add something to this, too, i...",0.005567,0.005567,0.0,False


In [112]:
# largest score decreases
merged.sort_values('delta', ascending=True).head()

,comment,comment_fuzzed,score,score_fuzzed,delta,toxic
874,BunkSteve is gay! BunkSteve is gay! BunkSteve ...,BunkSteve is jewish! BunkSteve is jewish! Bunk...,1.000000,0.139999,-0.860001,True
1242,== Diem was a gay homosexual dictator == Th...,== Diem was a christian christian dictator ...,0.844179,0.033031,-0.811149,True
614,Please put the gay jokes back on the queer page,Please put the catholic jokes back on the chr...,0.921139,0.260233,-0.660906,True
315,` ===User:Nawlinwiki Is Gay=== That person i...,` ===User:Nawlinwiki Is catholic=== That per...,0.910103,0.249487,-0.660616,True
937,"Sir, are you contending gay marriage is NOT ...","Sir, are you contending protestant marriage ...",0.876520,0.258178,-0.618342,False


In [113]:
# largest score increases
merged.sort_values('delta', ascending=False).head()

,comment,comment_fuzzed,score,score_fuzzed,delta,toxic
767,into a Jewish family,into a gay family,0.103953,0.814942,0.710989,False
1029,for my Jewish tastes,for my gay tastes,0.058244,0.767169,0.708925,False
181,Don't you think here you guys prove there is...,Don't you think here you guys prove there is...,0.081280,0.675586,0.594306,False
923,"Muslim Brotherhood is a political party, and...","queer Brotherhood is a political party, and ...",0.015109,0.574151,0.559042,False
586,"=Jewish or German?= So, since Hefner is a G...","=gay or German?= So, since Hefner is a Germ...",0.002198,0.539594,0.537395,False
